### 7.1. Автоматизация отчетности
##### Задача: наладить автоматическую отправку аналитической сводки в телеграм каждое утро. 
Для этого: 
1. Создайте телеграм-бота с помощью @BotFather
2. Напишите скрипт для сборки отчета по ленте новостей. Отчет должен состоять из двух частей:
* текст с информацией о значениях ключевых метрик за предыдущий день
* график с значениями метрик за предыдущие 7 дней
* Отобразите в отчете следующие ключевые метрики: 
    * DAU 
    * Просмотры
    * Лайки
    * CTR
3. Автоматизируйте отправку отчета с помощью Airflow.

In [ ]:
from datetime import date, datetime, timedelta # импорт даты, чтобы взять вчерашнюю дату

import pandas as pd
import io
import pandahouse as ph

import telegram
import matplotlib.pyplot as plt
import seaborn as sns 

from airflow.decorators import dag, task 
#from airflow.operators.python import get_current_contex


#для подключения к clickhouse
connection = {
              'host':'https://clickhouse.lab.karpov.courses',
              'database':'simulator_20221020',
              'user':'student', 
              'password':'dpo_python_2020'
             }


# Дефолтные параметры, которые прокидываются в таски
default_args = {
                'owner': 'i-aksakal-12',
                'depends_on_past': False,
                'retries': 2,
                'retry_delay': timedelta(minutes=5),
    
                'email': 'bailukova.bk@gmail.com', # Почта для уведомлений
                'email_on_failure': 'bailukova.bk@gmail.com', # Почта для уведомлений при ошибке
                'email_on_retry': 'bailukova.bk@gmail.com', # Почта для уведомлений при перезапуске
    
                'start_date': datetime(2022, 11, 9), # старт дата, с которой собираем данные9/11/2022
                }

# Интервал запуска DAG
schedule_interval = '0 11 * * *' #запускаем даг каждый день в 11 часов


# данные для подключения к боту
my_token = 'token' # заменила настоящий токен, поскольку репозиторий публичный
bot = telegram.Bot(token=my_token) # получаю доступ к бот
chat_id = id_number # заменила настоящий id, поскольку репозиторий публичный



@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def aksakal_bot_report():    
    
    @task()
    #1 выгружаю данные из базы - сразу рассчитываю метрики за последних 7 дней
    def extract_feed():
        query  = '''SELECT 
                       count(distinct user_id) as dau, 
                       toDate(time) as event_date,
                       sum(action = 'like') as likes, 
                       sum(action = 'view') as views, 
                       round((likes /views) * 100, 2) as ctr
                   FROM 
                       {db}.feed_actions 

                   group by
                        event_date
                   HAVING 
                       toDate(time) >= today() - 7 and toDate(time) <= today() - 1
                       order by event_date
                 '''
        df_cube_feed = ph.read_clickhouse(query, connection=connection)
        
        return df_cube_feed

    
    # создаю новый датафрейм, чтобы построить 1 график с likes/views
    @task()
    def transform_for_lineplot(df_cube_feed):
       
        df_likes = df_cube_feed[['event_date', 'likes']]
        df_likes['action'] = 'likes'
        df_likes.columns = ['event_date', 'value', 'action']

        df_views = df_cube_feed[['event_date', 'views']]
        df_views['action'] = 'views'
        df_views.columns = ['event_date', 'value', 'action']

        df_action = pd.concat([df_likes, df_views]).reset_index()
        
        return df_action

    
    @task()
    def load_bot_report(df_cube_feed, df_action):
        df= df_cube_feed.iloc[-1] # получаем данные за вчера для текстового отчета ВАРИАНТ 1
        #df = df_cube_feed[df_cube_feed.event_date == max(df_cube_feed.event_date)] # получаем данные за вчера для текстового отчета ВАРИАНТ 2
        
        # отправляем текстовое сообщение #\n - для переноса строки
        report_msg = "Дата: " + df['event_date'].strftime("%d.%m.%Y")+\
                    "\nDAU: " + str(df['dau']) +\
                    "\nLikes: " + str(df['likes']) +\
                    "\nViews: " + str(df['views']) +\
                    "\nCTR: " + str(df['ctr']) + "%"
        bot.sendMessage(chat_id=chat_id, text=report_msg)
    

        #рисую графики
        fig, axes = plt.subplots(3, 1, figsize=(10, 15))    # задаю размеры
        fig.suptitle("Ключевые метрики за последние 7 дней")   # общее название графиков

        axes[0].set(title='DAU')
        axes[0].set(xlabel='date', ylabel='DAU') 
        sns.lineplot(data=df_cube_feed, ax=axes[0], x='event_date', y='dau')
        
        axes[1].set(title='Likes and views')
        axes[1].set(xlabel='date', ylabel='likes/views')
        sns.lineplot(data=df_action, ax=axes[1], x="event_date", y="value", hue="action") # деление hue="action" на likes/views
        
        axes[2].set(title='CTR')
        axes[2].set(xlabel='date', ylabel='CTR')
        sns.lineplot(data=df_cube_feed, ax=axes[2], x='event_date', y='ctr')

        #сохраняю график в буфере
        plot_object = io.BytesIO()
        fig.savefig(plot_object)
        plot_object.seek(0) # перенос курсора в начало
        plot_object.name = 'main_metrics.png'
        plt.close()

        #отправляю график
        bot.sendPhoto(chat_id=chat_id, photo=plot_object)
        
    
    df_cube_feed = extract_feed()
    df_action = transform_for_lineplot(df_cube_feed)
    load_bot_report(df_cube_feed, df_action)

    
aksakal_bot_report = aksakal_bot_report()